In [1]:
from spacy.en import English
nlp = English()

In [165]:
import time as tm
import numpy as np
import pandas as pd
import re

verbose = False
np.random.seed(14)
input_csv = "data/snippets.csv"
full = pd.read_csv(input_csv) # , header=None)
print full.shape

full = full.dropna(axis=0)
print full.shape

full = full[full['Content'].str.len() < 600]
print full.shape

full = full[0:10000]
print full.shape

(65499, 3)
(64458, 3)
(64434, 3)
(10000, 3)


In [170]:
preX = full['Content']
preX = preX.apply(lambda row: row.strip())
preX = preX.apply(lambda row: row.replace('-', ' '))
parsed = preX.apply(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
reindexed = parsed.reset_index(drop=True)

print len(reindexed)

postX = []
for idx, sentence in enumerate(preX):
    found_entities = []
    for ent in reindexed[idx].ents:
        if ent.label_ == 'ORG':
            found_entities.append( (ent.text.encode('utf-8'), 'ORG') )
        if ent.label_ == 'PERSON':
            found_entities.append( (ent.text.encode('utf-8'), 'PERSON') )
        if ent.label_ == 'MONEY':
            found_entities.append( (ent.text.encode('utf-8'), 'MONEY') )
    for word in found_entities:
        sentence = sentence.replace(word[0], word[1])
    postX.append(sentence)
    if (idx%2500 == 0):
        print idx

10000
0
2500
5000
7500


In [171]:
postX = pd.Series(postX)
parsed = postX.map(lambda row: nlp(unicode(row, 'utf-8', 'ignore')) )
tokenized = parsed.map(lambda row: [token.orth_ for token in row])

X = tokenized
print len(X)

10000


In [173]:
y = full['Tags']

y = y.str.split(';').str[0]
print y.value_counts()

strategy                  1502
funding                   1390
key_person_hire            825
partnership                818
product                    721
acquisition                668
revenue                    662
overview                   447
users                      425
headcount                  375
award                      368
valuation                  198
turmoil                    170
founding                   167
customers                  156
market                     154
geo                        140
key_person_departure       131
total-funding              121
sector                     101
conference-sponsorship      75
ipo                         74
executive-promotion         64
competitors                 57
event-participant           52
office_move                 44
lawsuit                     35
reorg                       23
patent                      17
bankruptcy                  10
team-grew                    3
filing-ipo                   2
executiv

In [174]:
y = y.str.replace('total-', '')
y = y.str.replace('filing-', '')
y = y.str.replace('key_person_', '')

y = y.str.replace('lawsuit', 'OTHER')
y = y.str.replace('office_move', 'OTHER')
y = y.str.replace('geo', 'OTHER')
y = y.str.replace('reorg', 'OTHER')
y = y.str.replace('patent', 'OTHER')
y = y.str.replace('conference-sponsorship', 'OTHER')
y = y.str.replace('executive-promotion', 'OTHER')
y = y.str.replace('bankruptcy', 'OTHER')
y = y.str.replace('event-participant', 'OTHER')
y = y.str.replace('scale', 'OTHER')
y = y.str.replace('team-grew', 'OTHER')
y = y.str.replace('executive-publication', 'OTHER')
y = y.str.replace('job-posting', 'OTHER')
y = y.str.replace('leadership-change', 'OTHER')

y = y.fillna('OTHER')
jake = y.value_counts()
print len(jake)
print jake

21
funding        1511
strategy       1502
hire            825
partnership     818
product         721
acquisition     668
revenue         662
OTHER           468
overview        447
users           425
headcount       375
award           368
valuation       198
turmoil         170
founding        167
customers       156
market          154
departure       131
sector          101
ipo              76
competitors      57
dtype: int64


In [179]:
y = y.reset_index(drop=True)

9990     strategy
9991    valuation
9992      revenue
9993      revenue
9994     strategy
9995      funding
9996     strategy
9997      revenue
9998       sector
Name: Tags, dtype: object
9990    [The, ORG, currently, costs, $, MONEY, ,, but,...
9991    [ORG, pegged, the, football, organization, 's,...
9992    [The, Vikings, brought, in, an, estimated, $, ...
9993    [That, 's, compared, to, MONEY, in, revenue, a...
9994    [The, organization, enjoyed, a, major, boost, ...
9995    [The, makers, of, Vessyl, ,, the, intelligent,...
9996    [PERSON, ,, co, founded, by, PERSON, and, indu...
9997    [The, company, also, said, it, had, passed, MO...
9998    [After, mainstream, social, networks, began, c...
dtype: object


In [182]:
for i in xrange(10):
    print y[190+i]
    print X[190+i]

revenue
[u'The', u'net', u'proceeds', u'(', u'including', u'promoted', u'interests', u')', u'to', u'PERSON', u'were', u'MONEY', u',', u'which', u'equates', u'to', u'an', u'approximate', u'$', u'35', u'million', u'cash', u'profit', u'on', u'these', u'investments', u'.']
users
[u'Elite', u'Daily', u'had', u'20.6', u'million', u'unique', u'visitors', u'in', u'December', u'.']
users
[u'This', u'represents', u'a', u'38', u'%', u'decrease', u'from', u'January', u'of', u'the', u'same', u'year', u'.']
users
[u'ORG', u'.', u'(', u'ORG', u',', u'ORG', u':', u'ORG', u')', u',', u'a', u'leading', u'provider', u'of', u'security', u'and', u'monetization', u'solutions', u'that', u'enable', u'service', u'providers', u'to', u'protect', u'and', u'personalize', u'the', u'digital', u'experience', u',', u'announced', u'today', u'that', u'it', u'has', u'received', u'orders', u'from', u'four', u'existing', u'tier', u'1', u'mobile', u'operator', u'customers', u'for', u'Allot', u'Smart', u'Data', u'Source', u'

In [176]:
print y.shape
y.name="Tags"
print X.shape
data = pd.concat([X, y], axis=1)
print data.shape

(10000,)
(10000,)
(10131, 2)


In [25]:
print data['Content'][8771]
print data['Content'][10731]
print data['Content'][3451]
#  Tricky token parsing: [10731, 37877, 8771, 3451, 42795]

[u'[', u'Update', u'3:30pm', u'PST', u':', u'Another', u'source', u'now', u'tells', u'us', u'LiveRail', u'sold', u'for', u'$', u'500', u'million', u',', u'matching', u'the', u'$', u'400', u'million', u'to', u'$', u'500', u'million', u'range', u'I', u'reported', u'earlier', u'.', u']']
[u'\u201c', u'If', u'you', u'are', u'selling', u'a', u'product', u'for', u'$', u'12', u'and', u'you', u'see', u'your', u'competitor', u'is', u'selling', u'it', u'for', u'$', u'11.50', u'so', u'you', u'drop', u'your', u'price', u'to', u'$', u'11.49', u'you', u'have', u'not', u'accomplished', u'anything', u'except', u'starting', u'a', u'price', u'war', u',', u'\u201d', u'he', u'says', u'.']
[u'FirstBank', u"'s", u'net', u'income', u'for', u'full', u'-', u'year', u'2015', u'increased', u'to', u'$', u'177.9', u'million', u'from', u'$', u'175.4', u'million', u'in', u'2014', u'.']


In [32]:
data.to_pickle('data/training_data.p')